# Tensor & matrix factorization 2020 Final project

Here the code for decomposition algorithm is present

In [ ]:
import numpy as np
import tensorly as tl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
import torchvision

from torchsummary import summary

## Weights extraction from ResNet-18 (layer 4.1)

In [ ]:
resnet18 = torchvision.models.resnet18(pretrained=True)

In [ ]:
resnet18.cuda()
summary(resnet18, input_size=(3,224,224))

In [ ]:
for p in resnet18.named_parameters():
    print(p[0])

In [ ]:
Y = resnet18.layer4[1].conv1.weight.data.detach().cpu().clone().numpy()
Y = Y.reshape(512,512,9).transpose([2,0,1])
Y = np.transpose(Y, [1,2,0])

In [ ]:
Y.shape

In [ ]:
R = 50
R1, R2 = 30,30

In [ ]:
core.shape

## Attempts to implement [this article by AH Phan et al.](https://www.researchgate.net/publication/343626375_Stable_Low-rank_Tensor_Decomposition_for_Compression_of_Convolutional_Neural_Network)

In [ ]:
U = np.diag(np.ones(90))[:,:R1]
V = np.diag(np.ones(90))[:,:R2]
A = np.random.randn(R1,R)
B = np.random.randn(R2,R)
C = np.random.randn(9,R)

#Y_restored = np.transpose(tl.kruskal_to_tensor((np.ones(R), (U@A, V@B, C))), [0,1,2])

#print("Random tensors. Norm is", np.linalg.norm(Y-Y_restored))

In [ ]:
Y1 = tl.unfold(core,0)
for i in range(20):
    #print("Relative Norm is", np.linalg.norm((U@A @ (tl.tenalg.khatri_rao([C, V@B])).T) - Y1)/\
         # np.linalg.norm(Y1))
    A = U.T @ tl.unfold(core,0) @ (tl.tenalg.khatri_rao([C, V@B])) @ np.linalg.inv(C.T@C * (B.T@B))
    Q = tl.unfold(core,0) @ (tl.tenalg.khatri_rao([C, V@B]))@ np.linalg.inv((C.T@C * (B.T@B))) @\
        tl.tenalg.khatri_rao([C, V@B]).T @ tl.unfold(core,0).T

    eigvals, eigvecs = np.linalg.eigh(Q)
    indices = np.argsort(np.abs(eigvals))[-R1:]
    
    U = eigvecs[:,indices]
    
    B = V.T @ tl.unfold(core,1) @ (tl.tenalg.khatri_rao([C, U@A])) @ np.linalg.inv(C.T@C * (A.T@A))
    """
    R_ = tl.unfold(core,1) @ tl.tenalg.khatri_rao([C, U@A]) @ np.linalg.inv(C.T@C * (A.T@A)) @ \
        tl.tenalg.khatri_rao([C, U@A]).T@tl.unfold(core,1).T
    eigvals, eigvecs = np.linalg.eig(R_)
    indices = np.argsort(np.abs(eigvals))[-R2:]
    V = eigvecs[:,indices]
    
    C = tl.unfold(core,2) @ (tl.tenalg.khatri_rao([U@A, V@B])) @ np.linalg.inv(B.T@B * (A.T@A))
    #C[C < 0] = 0
    
    """
    
   

    
    
    print("Relative Norm is", np.linalg.norm((U@A @ (tl.tenalg.khatri_rao([C, V@B])).T) - Y1)/\
          np.linalg.norm(Y1))

Unfortunately this code did not converge in terms of norm of difference of restored and original tensor. Therefore I switched to less sophisticated version of TKD_CPD compression.

# TKD-CPD

In [ ]:
from tensorly.decomposition import partial_tucker

In [ ]:
Y = resnet18.layer4[1].conv1.weight.data.detach().cpu().clone().numpy()
Y = Y.reshape(512, 512, 9)
Y = Y.transpose((2, 1, 0))

In [ ]:
Y.shape

The function ```conduct_compression(Y, r1, r2, R)``` conducts tensor decompositions and saves results to corresponding binary ```*.npy``` files. There is no need to launch it as all decompositions were saved to [google drive](https://drive.google.com/drive/folders/11N0KO7ooM6pZomPOz9BiY6pq1deBJbwO).

Next cells are written to show which ranks combinations were computed.

In [ ]:
def conduct_compression(Y, r1,r2,R):
    core, factors = partial_tucker(Y, 
                                   modes=[1,2],
                                   ranks=[r1,r2],
                                   n_iter_max=200, 
                                   tol=1e-6)
    np.save(f"core_{r1}_{r2}_{R}", core)
    print("core:", core.shape)
    for i in range(len(factors)):
        np.save(f"factor_{i}_{r1}_{r2}_{R}" , factors[i])
        print("fi",factors[i].shape)
    core_weights, core_factors = tl.decomposition.parafac(core,
                             rank=R, 
                             n_iter_max=5000,
                             tol=1e-7)
    #print(type(core_factors), len(core_factors))
    np.save(f"core_weights_{r1}_{r2}_{R}", core_weights)
    #print("core weights:", core_weights.shape)
    for i in range(len(core_factors)):
        print("core fac", core_factors[i].shape)
        np.save(f"core_factor_{i}_{r1}_{r2}_{R}", core_factors[i])
        #print("core fact:", core_factors[i].shape)
    restored_core = tl.kruskal_to_tensor((core_weights, core_factors))
    #print("rest core:", restored_core.shape)
    np.linalg.norm(core-restored_core)/np.linalg.norm(core)

    restored_conv = np.zeros_like(Y)
    restored_conv = np.zeros_like(Y)
    for i in range(9):
        restored_conv[i] = tl.tucker_to_tensor((restored_core[i], factors))

    return np.linalg.norm(restored_conv-Y)/np.linalg.norm(Y)

In [ ]:
norms = {}
for R in [100]:
    norm = conduct_compression(Y, 90,80,R)
    norms[R] = norm
norms_aggr["90 90"] = norms

In [ ]:
norms = {}
for R in [100, 120, 150, 180, 210, 240, 250, 280, 300, 330, 380, 400]:
    norm = conduct_compression(Y, 90,90,R)
    norms[R] = norm
norms_aggr["90 90"] = norqms

In [ ]:
for r1 in (60,):
    print()
    print("r1 is ", r1, end="")
    for r2 in (30,60,90):
        print(r2, end = " ")
        for R in [100, 120, 150, 180, 240, 250]:
            norm = conduct_compression(Y, r1, r2,R)
            norms[R] = norm
        norms_aggr[str(r1) +" "+str(r2)] = norms

To parse decompositions, you should download ```npy``` files and place them to the folder ```decompositions``` in which is located in the same dir with current ```ipynb```.

The code below first checks if all decompositions from the ranks combination are present in the folder ```decompositions```.

In [ ]:
for r1 in (30,60,90):
    for r2 in (30,60,90):
        for R in [100, 120, 150, 180, 240, 250]:
            assert os.path.exists(f"./decompositions/core_{r1}_{r2}_{R}.npy")
            print(r1,r2,R)
            for i in range(2):
                print(i)
                assert os.path.exists(f"./decompositions/factor_{i}_{r1}_{r2}_{R}.npy")
            assert os.path.exists(f"./decompositions/core_weights_{r1}_{r2}_{R}.npy")
            for i in range(3):
                assert os.path.exists(f"./decompositions/core_factor_{i}_{r1}_{r2}_{R}.npy")



If the code above is successfully launched, we can then proceed to reconstruction of tensors. Moreover, we can launch then the next ```ipynb``` which is called ```resnet.ipynb```. It will parse decompositions and replace conv layers in the original NN with their compressed convs with corresponding kernels.

In [ ]:
core = np.load(f"./decompositions/core_{r1}_{r2}_{R}.npy")
factors = []

for i in range(3):
        factors.append(np.load(f"./decompositions/factor_{i}_{r1}_{r2}_{R}.npy"))

core_weights = np.load(f"./decompositions/core_weights_{r1}_{r2}_{R}.npy")
core_factors = []

for i in range(2):
    core_factors.append(np.load(f"./decompositions/core_factor_{i}_{r1}_{r2}_{R}.npy"))
core_factors = tuple(core_factors)
restored_core = tl.kruskal_to_tensor((core_weights, core_factors))
print(restored_core.shape)
    #np.linalg.norm(core-restored_core)/np.linalg.norm(core)
restored_conv = np.zeros_like(Y)
print(restored_conv.shape)

for i in range(9):
    print(i)
    restored_conv[i] = tl.tucker_to_tensor((restored_core[i], factors))

In [ ]:
Y.shape

Below you can see the simple function which counts the compression rate, i.e. ratio of n_parameters in the original layers to the number of them in compressed kernels.

In [ ]:
def CompressionRate(r1,r2,R):
    s = 9*r1*r2 + 512*(r1+r2) + 9*R + R*(r1+r2)
    init = 512*512*9.
    return init/s

In [ ]:
CompressionRate(90, 90, 100)